In [1]:
import pandas as pd
import numpy as np
import psycopg2 as psy
import query_to_redshift as qtr

In [2]:
original_df_up = pd.read_csv('../raw_data/Excel_files/Item_margin_up_pre_campaign_202106.csv', index_col=0)
original_df_down = pd.read_csv('../raw_data/Excel_files/Item_margin_down_pre_campaign_202106.csv', index_col=0)

In [3]:
homie_days_query = """
SELECT item_code,
       MIN(period_date),
       MAX(period_date),
       webshop_code,
       final_price_amount,
       item_campaign_key,
       item_campaign_label,
       item_campaign_category
  from public.item_catalog_history
 where item_campaign_category like '%HomieDay%'
   and item_campaign_key like '%14%'
   and sellable_flag = true
   and webshop_code = 'DE'
   and period_date >= '2021-04-29'
 GROUP BY item_code, webshop_code, final_price_amount,
 item_campaign_label, item_campaign_key, item_campaign_category
HAVING (MAX(period_date) - MIN(period_date)) > 14."""


In [4]:
homie_bazooka_query = """
SELECT item_code,
       MIN(period_date) as min_date,
       MAX(period_date) as max_date,
       --webshop_code,
       MAX(final_price_amount) as max_price,
       MAX(item_campaign_key) as max_item_campaign_key,
       MAX(item_campaign_label) as max_item_campaign_label,
       MAX(item_campaign_category) as max_item_campaign_category
  from public.item_catalog_history
 where item_campaign_category like '[CAMPAIGN]HomieDay_Voucher_CW17|%'
   and item_campaign_key like 'party14%'
   and sellable_flag = true
   and webshop_code = 'DE'
   and period_date >= '2021-04-29'
 GROUP BY item_code
HAVING MIN(period_date) = '2021-04-29'
   and MAX(period_date) = '2021-05-10';"""


In [5]:
#df_homie14 = qtr.redshift_query_to_dataframe(qtr.cursor_redshift, homie_days_query)
df_homie14 = qtr.redshift_query_to_dataframe(qtr.cursor_redshift, homie_bazooka_query)

In [6]:
print(len(df_homie14.index))

7335


In [7]:
print(df_homie14.head())

            item_code    min_date    max_date                max_price  \
0  000000001000054997  2021-04-29  2021-05-10   749.990000000000000000   
1  000000001000151822  2021-04-29  2021-05-10   164.990000000000000000   
2  000000001000186860  2021-04-29  2021-05-10  1549.990000000000000000   
3  000000001000144471  2021-04-29  2021-05-10   159.990000000000000000   
4  000000001000003310  2021-04-29  2021-05-10    77.990000000000000000   

          max_item_campaign_key           max_item_campaign_label  \
0  party14|3dmodels|testcynthia  EXTRA -14 %|3dmodels|testcynthia   
1              party14|24series              EXTRA -14 %|EXKLUSIV   
2                party14|40sale                    EXTRA -14 %|40   
3                party14|30sale                    EXTRA -14 %|30   
4                party14|50sale                  EXTRA -14 %|Sale   

                    max_item_campaign_category  
0  [CAMPAIGN]HomieDay_Voucher_CW17|2019032601|  
1             [CAMPAIGN]HomieDay_Voucher_C

In [8]:
df_homie14.dtypes

item_code                     object
min_date                      object
max_date                      object
max_price                     object
max_item_campaign_key         object
max_item_campaign_label       object
max_item_campaign_category    object
dtype: object

In [9]:
df_homie14 = df_homie14.astype({'item_code': int})

In [10]:
df_homie14.dtypes

item_code                      int64
min_date                      object
max_date                      object
max_price                     object
max_item_campaign_key         object
max_item_campaign_label       object
max_item_campaign_category    object
dtype: object

In [11]:
df_homie14.head()


,item_code,min_date,max_date,max_price,max_item_campaign_key,max_item_campaign_label,max_item_campaign_category
0,1000054997,2021-04-29,2021-05-10,749.990000000000000000,party14|3dmodels|testcynthia,EXTRA -14 %|3dmodels|testcynthia,[CAMPAIGN]HomieDay_Voucher_CW17|2019032601|
1,1000151822,2021-04-29,2021-05-10,164.990000000000000000,party14|24series,EXTRA -14 %|EXKLUSIV,[CAMPAIGN]HomieDay_Voucher_CW17|
2,1000186860,2021-04-29,2021-05-10,1549.990000000000000000,party14|40sale,EXTRA -14 %|40,[CAMPAIGN]HomieDay_Voucher_CW17|
3,1000144471,2021-04-29,2021-05-10,159.990000000000000000,party14|30sale,EXTRA -14 %|30,[CAMPAIGN]HomieDay_Voucher_CW17|
4,1000003310,2021-04-29,2021-05-10,77.990000000000000000,party14|50sale,EXTRA -14 %|Sale,[CAMPAIGN]HomieDay_Voucher_CW17|


In [12]:
merged_down = pd.merge(original_df_down, df_homie14, on='item_code')


In [13]:
merged_up = pd.merge(original_df_up, df_homie14, on='item_code')

In [14]:
merged_up.head()


,item_code,item_price_x,max_date_x,delivery_weeks,sales_pe,sales_pe_err,beta_del_week,beta_del_week_err,intercept,intercept_err,...,N_prime_sales,Delta_Margin,Group_flag,Increment,min_date,max_date_y,max_price,max_item_campaign_key,max_item_campaign_label,max_item_campaign_category
0,1000118440,74.99,2021-04-28,1,-8.4735,2.2546,-0.3889,0.1521,46.9114,11.1327,...,8.829126,3.875773,up,14.000000000000002%,2021-04-29,2021-05-10,74.990000000000000000,party14|40sale,EXTRA -14 %|40,[CAMPAIGN]HomieDay_Voucher_CW17|
1,1000086540,349.99,2021-04-28,3,-1.4374,0.2356,-0.0499,0.0098,9.5983,1.4798,...,0.840108,0.778171,up,14.000000000000002%,2021-04-29,2021-05-10,349.990000000000000000,party14|3dmodels,EXTRA -14 %|3dmodels,[CAMPAIGN]HomieDay_Voucher_CW17|2019032601
2,1000050876,569.99,2021-04-18,2,-1.1908,0.3369,-0.0119,0.0089,9.2930,2.1853,...,1.556809,5.774286,up,14.000000000000002%,2021-04-29,2021-05-10,569.990000000000000000,party14|24series,EXTRA -14 %|EXKLUSIV,[CAMPAIGN]HomieDay_Voucher_CW17|
3,1000209625,194.99,2021-04-28,11,-2.3602,0.4350,-0.0348,0.0131,15.1122,2.7039,...,1.974935,1.731161,up,14.000000000000002%,2021-04-29,2021-05-10,194.990000000000000000,party14|30sale|storage,EXTRA -14 %|30|storage,[CAMPAIGN]HomieDay_Voucher_CW17||
4,1000177730,329.99,2021-04-28,4,-1.0609,0.2503,-0.0078,0.0053,7.0730,1.5811,...,0.750567,1.220691,up,14.000000000000002%,2021-04-29,2021-05-10,329.990000000000000000,party14|storage,EXTRA -14 %|storage,[CAMPAIGN]HomieDay_Voucher_CW17|


In [15]:
merged_down.head()

,item_code,item_price_x,max_date_x,delivery_weeks,sales_pe,sales_pe_err,beta_del_week,beta_del_week_err,intercept,intercept_err,...,N_prime_sales,Delta_Margin,Group_flag,Increment,min_date,max_date_y,max_price,max_item_campaign_key,max_item_campaign_label,max_item_campaign_category
0,1000048003,349.99,2021-04-23,3,-2.7317,0.3851,-0.0089,0.0067,16.2329,2.1005,...,0.616165,0.590100,down,-14.000000000000002%,2021-04-29,2021-05-10,349.990000000000000000,party14|storage,EXTRA -14 %|storage,[CAMPAIGN]HomieDay_Voucher_CW17|
1,1000129691,214.99,2021-01-10,2,-2.2732,0.2517,-0.0556,0.0139,11.4418,1.1915,...,-0.534978,1.743469,down,-14.000000000000002%,2021-04-29,2021-05-10,229.990000000000000000,party14|50sale,EXTRA -14 %|Sale,[CAMPAIGN]HomieDay_Voucher_CW17|
2,1000123695,199.99,2021-04-28,1,-2.5115,0.4648,-0.0307,0.0166,13.1606,2.3216,...,0.202093,1.547909,down,-14.000000000000002%,2021-04-29,2021-05-10,199.990000000000000000,party14|24series|storage,EXTRA -14 %|EXKLUSIV|storage,[CAMPAIGN]HomieDay_Voucher_CW17||
3,1000128456,429.99,2021-04-28,3,-0.7598,0.2379,-0.0255,0.0155,4.7650,1.3544,...,0.195849,0.222921,down,-14.000000000000002%,2021-04-29,2021-05-10,429.990000000000000000,party14|eurocup|eurocup21|24series,EXTRA -14 %|Euro Cup|Euro Cup|EXKLUSIV,[CAMPAIGN]HomieDay_Voucher_CW17|||
4,1000091262,289.99,2020-10-14,3,-3.3256,0.4310,-0.1002,0.0091,18.4672,2.3485,...,-0.187465,2.056987,down,-14.000000000000002%,2021-04-29,2021-05-10,319.990000000000000000,party14|24series,EXTRA -14 %|EXKLUSIV,[CAMPAIGN]HomieDay_Voucher_CW17|


In [16]:
list_of_items_up = merged_up['item_code'].to_numpy()

In [17]:
list_of_items_down = merged_down['item_code'].to_numpy()


In [18]:

print(list_of_items_up)

[1000118440 1000086540 1000050876 1000209625 1000177730 1000209641
 1000184191 1000187602 1000027369 1000110334 1000008842 1000092019
 1000092055 1000118147 1000103648 1000121202 1000056978 1000008016
 1000209640 1000104115 1000189778 1000136114 1000209610 1000103647
 1000164014 1000205104 1000207389 1000189771 1000177767 1000209634
 1000050865 1000117846 1000056783 1000008872 1000039246 1000025086
 1000091560 1000015909 1000136205 1000056964 1000025087 1000183841
 1000056953 1000086543 1000110022 1000033524 1000216583 1000076797
 1000194574 1000209631 1000050842 1000056782 1000152213 1000008037
 1000129379 1000119769 1000008035 1000052657 1000123395 1000210971]


In [19]:
print(list_of_items_down)

[1000048003 1000129691 1000123695 1000128456 1000091262 1000145854
 1000123763 1000110348 1000056742 1000087664 1000071065 1000008825
 1000200152 1000024953 1000086500 1000007069 1000024947 1000151788
 1000117189 1000007079 1000008056 1000039261 1000210541 1000050883
 1000008867 1000008752 1000008808 1000118200 1000113650 1000022120
 1000033528 1000131866 1000027408 1000127796 1000126227 1000007429
 1000008058 1000007430 1000112303 1000005213 1000197490 1000112266
 1000007080 1000153429 1000022436 1000113296 1000187570 1000213239
 1000008748 1000151250 1000006442 1000008007 1000126218 1000064654
 1000125642 1000008755 1000034491 1000007107 1000050873 1000040375
 1000110374 1000117190 1000175476 1000209624 1000027410 1000008814
 1000008031 1000210654 1000147149 1000008815 1000149746 1000007083
 1000181292 1000183858 1000104291 1000027357 1000008067 1000005178
 1000206722 1000134697 1000022129 1000008824 1000092132 1000008809
 1000050870 1000022466 1000134688 1000123195 1000126687 100000

In [20]:
merged_up['item_code'].to_csv('../raw_data/Excel_files/list_items_up_bazooka.csv')
merged_down['item_code'].to_csv('../raw_data/Excel_files/list_items_down_bazooka.csv')